# Fake News Detection using Logistic Regression

Fake news has become a major problem in today’s digital world, affecting politics, health, and public safety. This project focuses on detecting fake news using machine learning techniques, specifically logistic regression. We process the text data using NLP tools and build a model to classify articles as either real or fake.

About the Dataset:

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks whether the news article is real or fake:
           1: Fake news
           0: real News

#Importing the dependencies

In [50]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [51]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [52]:
#printing the stopwords in english
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

#Data Preprocessing

In [53]:
#loading the dataset into pandas dataframe
news_dataset=pd.read_csv('/content/news.csv')

In [54]:
news_dataset.shape

(20800, 5)

In [55]:
#printing the first 5 rows of the dataset
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [56]:
#Counting the no. of missing values in the dataset
news_dataset.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [57]:
#Filling the missing values with empty string
news_dataset=news_dataset.fillna('')

In [58]:
#combining the title and the author column
news_dataset['content']=news_dataset['author']+' '+news_dataset['title']

In [59]:
news_dataset

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"Alex Ansary NATO, Russia To Hold Parallel Exer..."


#Stemming

Stemming is the process of reducing a word to its Root word

example:
actor,actress,acting -->act

In [60]:
port_stem=PorterStemmer()

In [61]:
x='The Sun Is Shining.'
stemmed_content=re.sub('[^a-zA-Z]'," ",x)
print(stemmed_content)
stemmed_content=stemmed_content.lower()
print(stemmed_content)
stemmed_content=stemmed_content.split()
print(stemmed_content)
stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]

The Sun Is Shining 
the sun is shining 
['the', 'sun', 'is', 'shining']


In [62]:
def stemming (content):
  stemmed_content=re.sub('[^a-zA-Z]'," ",content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return(stemmed_content)

In [48]:
news_dataset['content']=news_dataset['content'].apply(stemming)

In [49]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


#Separating Data and Label

In [66]:
X=news_dataset['content'].values
Y=news_dataset['label'].values

In [67]:
X,Y

(array(['Darrell Lucus House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It',
        'Daniel J. Flynn FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart',
        'Consortiumnews.com Why the Truth Might Get You Fired', ...,
        'Michael J. de la Merced and Rachel Abrams Macy’s Is Said to Receive Takeover Approach by Hudson’s Bay - The New York Times',
        'Alex Ansary NATO, Russia To Hold Parallel Exercises In Balkans',
        'David Swanson What Keeps the F-35 Alive'], dtype=object),
 array([1, 0, 1, ..., 0, 1, 1]))

In [68]:
Y.shape

(20800,)

In [71]:
#convering the textual data to numerical values
vectorizer=TfidfVectorizer()
vectorizer.fit(X)
X=vectorizer.transform(X)

In [72]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 273725 stored elements and shape (20800, 25173)>
  Coords	Values
  (0, 904)	0.26354004814013343
  (0, 3862)	0.30579573877221844
  (0, 4507)	0.20531415441295317
  (0, 5508)	0.2993429551929777
  (0, 5800)	0.2502787762405247
  (0, 6145)	0.24677171892553343
  (0, 7574)	0.23047267305353566
  (0, 10387)	0.1844880289323935
  (0, 11307)	0.1532265401605094
  (0, 11409)	0.20615188166061463
  (0, 12528)	0.24883399099107747
  (0, 12902)	0.3024224900242886
  (0, 19171)	0.22537992364975484
  (0, 22289)	0.3484071341454308
  (0, 22649)	0.26575278886038384
  (0, 23355)	0.18006497451107856
  (1, 2544)	0.28998438336643234
  (1, 3075)	0.1531053111853744
  (1, 3509)	0.3775183944330703
  (1, 4298)	0.1902428965987476
  (1, 5469)	0.26240126155666194
  (1, 8420)	0.7045992054867244
  (1, 10134)	0.18787145765749735
  (1, 15149)	0.1586226371149596
  (1, 23748)	0.29662102960192643
  :	:
  (20797, 17253)	0.2367895682882451
  (20797, 17550)	0.302371441532

#Spliting the data into training and test data

In [75]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

#Training the model: Logistic Regression

In [76]:
model=LogisticRegression()

In [77]:
model.fit(X_train,Y_train)

LogisticRegression()

#Evalution

##Accuracy Score

In [81]:
#Accuracy score on training data
X_train_prediction=model.predict(X_train)
accuracy=accuracy_score(X_train_prediction,Y_train)
print("Accuracy: ",accuracy)

Accuracy:  0.9870192307692308


In [83]:
#Accuracy score on test data
X_test_prediction=model.predict(X_test)
accuracy=accuracy_score(X_test_prediction,Y_test)
print("Accuracy: ",accuracy)

Accuracy:  0.9788461538461538


#Making a Predictive system

In [85]:
X_new=X_test[1]
predictor=model.predict(X_new)
if(predictor==0):
  print("The news is Real!")
else:
  print("The news is Fake!")

The news is Real!


In [86]:
print(Y_test[1])

0


## Conclusion

The logistic regression model performed well in distinguishing fake news from real news using TF-IDF features. The preprocessing steps like stemming and stopword removal significantly helped improve the text representation. Future work could include more advanced models and additional data sources.